## Carga del modelo HMM  previamente entrenado

In [1]:
# Cargamos las probabilidades del modelo HMM
import numpy as np
transitionProdbict = np.load('transitionHMM.npy', allow_pickle='True').item()
emissionProbdict = np.load('emissionHMM.npy', allow_pickle='True').item()

### 📌 ¿Qué hace esta celda?

Carga dos archivos de datos guardados previamente:
- **transitionProdbict**: Diccionario con probabilidades de transición entre etiquetas POS (Part of Speech)
  - Ejemplo: `{'NOUN|VERB': 0.45}` = probabilidad de ir de NOUN a VERB es 45%
- **emissionProbdict**: Diccionario con probabilidades de que una palabra tenga una etiqueta específica
  - Ejemplo: `{'gato|NOUN': 0.8}` = probabilidad de que "gato" sea un NOUN es 80%

Estos datos fueron entrenados previamente y guardados en formato NumPy para reutilizarlos aquí.

In [2]:
transitionProdbict

{'NOUN|DET': 0.7051234338171443,
 'PUNCT|NOUN': 0.21745022412175544,
 'ADP|PUNCT': 0.10090433127082342,
 'ADJ|ADP': 0.012436935351402088,
 'NOUN|ADJ': 0.21575826037842416,
 'ADP|NOUN': 0.32356926925883456,
 'SCONJ|ADP': 0.017834095975595446,
 'VERB|SCONJ': 0.09595070422535211,
 'VERB|VERB': 0.029147490685952224,
 'NUM|VERB': 0.020600482138943676,
 'NOUN|NUM': 0.5239130434782608,
 '_|NOUN': 0.05962681121651204,
 'ADP|_': 0.8613312202852615,
 'DET|ADP': 0.5141382142438109,
 'VERB|ADP': 0.057022175290390706,
 '_|VERB': 0.03550295857988166,
 'DET|NOUN': 0.014177004065464401,
 'CCONJ|NOUN': 0.04951527155217346,
 'ADJ|CCONJ': 0.08459422283356259,
 'ADP|ADJ': 0.24936458627506355,
 'NOUN|ADP': 0.19464977120732135,
 'ADJ|NOUN': 0.17043677681642866,
 'PUNCT|ADJ': 0.24795255577520475,
 'VERB|PUNCT': 0.06964937331429479,
 'ADP|VERB': 0.24479509094893712,
 'PRON|DET': 0.01786378861183476,
 'DET|PRON': 0.024761904761904763,
 'PROPN|DET': 0.12095273539263118,
 'PROPN|PROPN': 0.22665345553628932,
 'AD

### 📌 ¿Qué hace esta celda?

Muestra el contenido del diccionario `transitionProdbict`. Es una visualización para entender la estructura de los datos:
- Cada clave es un par de etiquetas separadas por `|`
- Cada valor es la probabilidad de transición de una a otra
- Ejemplo: `NOUN|VERB` significa "dado que estamos en NOUN, ¿cuál es la probabilidad de ir a VERB?"

In [3]:
stateSet = set([w.split('|')[1] for w in list(emissionProbdict.keys())])
stateSet

{'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SYM',
 'VERB',
 '_'}

### 📌 ¿Qué hace esta celda?

Extrae todas las etiquetas POS (categorías gramaticales) únicas del diccionario de emisión:
1. Recorre todas las claves del diccionario (formato: `palabra|ETIQUETA`)
2. Extrae la parte después del `|` para obtener solo la etiqueta
3. Las agrupa en un conjunto (set) eliminando duplicados
4. **Resultado**: conjunto con todas las categorías gramaticales disponibles (NOUN, VERB, ADJ, etc.)

In [4]:
tagStateDict = {}
for i, state in enumerate(stateSet):
    tagStateDict[state] = i
    
tagStateDict

{'SCONJ': 0,
 'PROPN': 1,
 'NOUN': 2,
 'NUM': 3,
 'PUNCT': 4,
 'INTJ': 5,
 '_': 6,
 'PRON': 7,
 'CCONJ': 8,
 'ADV': 9,
 'ADJ': 10,
 'AUX': 11,
 'DET': 12,
 'SYM': 13,
 'ADP': 14,
 'VERB': 15,
 'PART': 16}

### 📌 ¿Qué hace esta celda?

Crea un diccionario de mapeo entre etiquetas y índices numéricos:
- **Objetivo**: Convertir nombres de etiquetas (texto) a números (índices)
- **Uso**: Permite acceder fácilmente a filas específicas en matrices de probabilidad
- **Ejemplo**: `{'NOUN': 0, 'VERB': 1, 'ADJ': 2, ...}`
- Esto es necesario porque las matrices de NumPy se indexan con números, no con etiquetas de texto

## Distribucion inicial de estados latentes

In [5]:
initTagStateProb = {} # \rho_i^{(0)}
from conllu import parse_incr 
wordList = []
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8")
count = 0 # cuenta la longitud del corpus
for tokenlist in parse_incr(data_file):
  count += 1
  tag = tokenlist[0]['upos']
  if tag in initTagStateProb.keys():
    initTagStateProb[tag] += 1
  else:
    initTagStateProb[tag] = 1

for key in initTagStateProb.keys():
  initTagStateProb[key] /= count

initTagStateProb

{'DET': 0.36275695284159615,
 'PROPN': 0.1124546553808948,
 'ADP': 0.15538089480048367,
 'PRON': 0.06348246674727932,
 'SCONJ': 0.02418379685610641,
 'ADV': 0.056831922611850064,
 'PUNCT': 0.08222490931076179,
 'VERB': 0.02418379685610641,
 'ADJ': 0.010882708585247884,
 'CCONJ': 0.032648125755743655,
 'NOUN': 0.02720677146311971,
 '_': 0.009068923821039904,
 'INTJ': 0.0006045949214026602,
 'AUX': 0.016324062877871828,
 'NUM': 0.01995163240628779,
 'PART': 0.0018137847642079807}

### 📌 ¿Qué hace esta celda?

Calcula la **distribución inicial de probabilidades** (ρ₀ en la notación matemática):
1. Lee el archivo de entrenamiento `es_ancora-ud-dev.conllu` (corpus de oraciones etiquetadas)
2. Para cada oración, toma la etiqueta de la PRIMERA palabra
3. Cuenta cuántas veces aparece cada etiqueta como primer elemento
4. Divide cada conteo por el número total de oraciones
5. **Resultado**: Probabilidad de que cada categoría gramatical sea el inicio de una oración

**Ejemplo**: Si NOUN aparece al inicio en 230 de 1000 oraciones → P(NOUN al inicio) = 0.23

In [6]:
np.array([initTagStateProb[k] for k in initTagStateProb.keys()]).sum()

np.float64(1.0)

### 📌 ¿Qué hace esta celda?

**Verificación de normalización**: Suma todas las probabilidades iniciales.
- Debe dar exactamente **1.0** (o muy cercano a 1.0)
- Esto confirma que las probabilidades están correctamente normalizadas
- Si el resultado no es ~1.0, significa que hay un error en el cálculo anterior

## Construccion del modelo de viterbi

In [7]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 📌 ¿Qué hace esta celda?

Descarga los recursos necesarios de NLTK para procesar texto:
- `punkt`: Modelo de tokenización que divide el texto en palabras y oraciones
- **Necesario**: La función `word_tokenize()` usará este modelo para convertir una oración en lista de palabras

In [8]:
def ViterbiTags(secuencia, transitionProdbict = transitionProdbict, emissionProbdict = emissionProbdict, tagStateDict = tagStateDict, initTagStateProb = initTagStateProb):
    
    seq = word_tokenize(secuencia)
    viterbiProb = np.zeros((17, len(seq))) # matriz de probabilidades
    
    for key in tagStateDict.keys():
        tag_row = tagStateDict[key]
        word_tag = seq[0].lower() + '|' + key
        if word_tag in emissionProbdict.keys():
            viterbiProb[tag_row][0] = initTagStateProb[key] * emissionProbdict[word_tag]
            
    # computo prob. siguientes columnas
    for col in range(1, len(seq)):
        for key in tagStateDict.keys():
            tag_row = tagStateDict[key]
            word_tag = seq[col].lower() + '|' + key   
            if word_tag in emissionProbdict.keys():
                possible_probs = []
                for key2 in tagStateDict.keys():
                    tag_row2 = tagStateDict[key2]
                    tag_prevtag = key + '|' + key2
                    if tag_prevtag in transitionProdbict.keys():
                        if viterbiProb[tag_row2][col-1] > 0:
                            possible_probs.append(viterbiProb[tag_row2, col-1]*transitionProdbict[tag_prevtag]*emissionProbdict[word_tag])
                viterbiProb[tag_row][col] = max(possible_probs)  
    
    # Construccion de la secuancia de tags
    res = []
    for i, p in enumerate(seq):
        for tag in tagStateDict.keys():
            if tagStateDict[tag] == np.argmax(viterbiProb[:,i]):
                res.append((p, tag))
            
    return res


ViterbiTags("El gato come pescado")


[('El', 'DET'), ('gato', 'NOUN'), ('come', 'SCONJ'), ('pescado', 'SCONJ')]

### 📌 ¿Qué hace esta celda?

**Implementa el ALGORITMO DE VITERBI** para etiquetar palabras con sus categorías gramaticales (POS Tagging).

**Pasos del algoritmo:**

1. **Tokenización**: Convierte la entrada en lista de palabras
2. **Inicialización (columna 0)**:
   - Para cada etiqueta: calcula P(etiqueta) × P(palabra|etiqueta)
   - Llena la primera columna de la matriz de probabilidades

3. **Recurrencia (columnas 1 a n)**:
   - Para cada palabra y etiqueta:
     - Busca cuál es la etiqueta anterior más probable
     - Multiplica: P(etiqueta anterior) × P(transición) × P(palabra|etiqueta)
     - Toma el máximo de todas las opciones

4. **Backtracking (construcción del resultado)**:
   - Recorre todas las palabras
   - En cada posición, encuentra la etiqueta con mayor probabilidad
   - Retorna lista de tuplas (palabra, etiqueta predicha)

**Resultado**: Las palabras etiquetadas con sus categorías gramaticales más probables

In [9]:
ViterbiTags("estos instrumentos han de rasgar")

[('estos', 'DET'),
 ('instrumentos', 'NOUN'),
 ('han', 'AUX'),
 ('de', 'ADP'),
 ('rasgar', 'VERB')]

### 📌 ¿Qué hace esta celda?

**Segunda prueba del modelo Viterbi**: Etiqueta la oración "estos instrumentos han de rasgar"
- Verifica que el algoritmo funcione correctamente con otro ejemplo
- Muestra cómo el modelo predice la categoría gramatical de cada palabra
- Confirma que el modelo generaliza a nuevas oraciones no vistas durante el entrenamiento

## Entrenamiento directo de HMM con NLTK

In [10]:
nltk.download('treebank')
from nltk.corpus import treebank
train_data = treebank.tagged_sents()[:3900]

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


### 📌 ¿Qué hace esta celda?

Descarga el corpus **Treebank** de NLTK:
- Corpus de texto inglés profesionalmente etiquetado con POS tags
- Carga las primeras 3900 oraciones etiquetadas
- **Estructura**: Lista de listas de tuplas `[(palabra, etiqueta), (palabra, etiqueta), ...]`
- **Propósito**: Usar estos datos para entrenar un modelo HMM automáticamente
- Este es un método alternativo al modelo Viterbi entrenado previamente

In [11]:
train_data

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], ...]

### 📌 ¿Qué hace esta celda?

Muestra la estructura de los datos de entrenamiento:
- Visualiza cómo están organizadas las 3900 oraciones
- Cada oración es una lista de tuplas (palabra, etiqueta POS)
- Permite verificar que los datos se cargaron correctamente

In [12]:
from nltk.tag import hmm
tagger = hmm.HiddenMarkovModelTrainer().train_supervised(train_data)

### 📌 ¿Qué hace esta celda?

**Entrena automáticamente un modelo HMM** usando NLTK:
- NLTK calcula automáticamente las matrices de transición y emisión
- Aprende de las 3900 oraciones de entrenamiento
- **Retorna**: Un tagger listo para usar (`tagger`)
- **Ventaja**: No necesita implementar manualmente el algoritmo como hicimos con Viterbi
- Este es un método más directo y automático comparado con el modelo pre-entrenado que cargamos al inicio

In [13]:
tagger.tag("Pierre Vinken will get old".split())

d:\david\learning_platzi\machine_learning\algoritmos_clasificacion_texto\.venv\Lib\site-packages\nltk\tag\hmm.py:333: RuntimeWarning: overflow encountered in cast
  X[i, j] = self._transitions[si].logprob(self._states[j])
d:\david\learning_platzi\machine_learning\algoritmos_clasificacion_texto\.venv\Lib\site-packages\nltk\tag\hmm.py:335: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])
d:\david\learning_platzi\machine_learning\algoritmos_clasificacion_texto\.venv\Lib\site-packages\nltk\tag\hmm.py:331: RuntimeWarning: overflow encountered in cast
  P[i] = self._priors.logprob(si)


[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 ('will', 'MD'),
 ('get', 'VB'),
 ('old', 'JJ')]

### 📌 ¿Qué hace esta celda?

**Prueba el modelo HMM entrenado** en una oración en inglés:
- Input: `"Pierre Vinken will get old".split()` → lista de palabras
- Output: Lista de tuplas (palabra, etiqueta POS predicha)
- Demuestra que el modelo etiqueta correctamente palabras que vio durante entrenamiento
- Verifica el funcionamiento básico del tagger de NLTK

In [14]:
tagger.evaluate(train_data)

C:\Users\david\AppData\Local\Temp\ipykernel_11632\3521729379.py:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  tagger.evaluate(train_data)


0.9815403947224078

### 📌 ¿Qué hace esta celda?

**Evalúa la precisión del modelo HMM entrenado**:
- Calcula qué porcentaje de palabras el modelo etiqueta correctamente
- Prueba sobre los mismos datos de entrenamiento (train_data)
- **Métrica**: Accuracy = (predicciones correctas / total predicciones) × 100
- Sirve como prueba de qué tan bien el modelo aprendió durante el entrenamiento
- Típicamente el accuracy en datos de entrenamiento es alto (~90%+), pero en datos nuevos es menor